The policy f generic prescribing is frequently cited as a policy that has saved the NHS significant amount of money. One [estimate] puts a figure of £7.1billion of savings over the last 40 years. Here we seek to identify missed savings over the last few years. 

At OpenPrescribing our [PPU tool](https://openprescribing.net/all-england/price-per-unit/) identifies the wide variation in the unit cost of a number of medicines prescribed across England estimates what could be saved if every organisation were prescribing as well as the best 10%. WE will seek to isolate the brand -> generic savings only

### Library Imports

In [1]:
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [2]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Data Imports

In [3]:
sql = '''WITH generic_ppus AS (
  SELECT
    presentation.bnf_code,
    presentation.name,
    vw__median_price_per_unit.date,
    vw__median_price_per_unit.median_price_per_unit
  FROM
    measures.vw__median_price_per_unit
  INNER JOIN
    hscic.presentation
  ON
    vw__median_price_per_unit.bnf_code = presentation.bnf_code
  INNER JOIN
    dmd.vmpp
  ON vmpp.bnf_code = presentation.bnf_code
  INNER JOIN dmd.dtinfo
  ON vmpp.id = dtinfo.vmpp
  WHERE
    substr(presentation.bnf_code, 10, 2) = 'AA' -- generic
   AND dtinfo.pay_cat IN (1,3,11)),

branded_prescribing AS (
  SELECT
    *,
    CONCAT(SUBSTR(bnf_code, 1, 9), 'AA', SUBSTR(bnf_code, 12, 4)) AS generic_bnf_code
  FROM
    hscic.normalised_prescribing_standard
  WHERE
    substr(bnf_code, 10, 2) != 'AA' -- not generic
  )


SELECT
  CAST(generic_ppus.date AS date) AS date,
  generic_ppus.bnf_code,
  pres_stat,
  generic_ppus.name,
  generic_ppus.median_price_per_unit AS target_ppu,
  SUM(branded_prescribing.net_cost) AS cost,
  SUM(branded_prescribing.quantity) AS quantity,
  SUM(branded_prescribing.quantity) * generic_ppus.median_price_per_unit AS target_cost,
  SUM(branded_prescribing.net_cost) - SUM(branded_prescribing.quantity) * generic_ppus.median_price_per_unit AS saving
FROM
  generic_ppus
INNER JOIN
  branded_prescribing
ON
  generic_ppus.bnf_code = branded_prescribing.generic_bnf_code
  AND generic_ppus.date = branded_prescribing.month
INNER JOIN
ebmdatalab.dmd.vmp AS vmp
ON
generic_ppus.bnf_code = vmp.bnf_code
GROUP BY
  generic_ppus.date,
  generic_ppus.bnf_code,
  pres_stat,
  generic_ppus.name,
  target_ppu
ORDER BY
  saving DESC'''

df_generic_saving = bq.cached_read(sql, csv_path='generic_saving.csv', use_cache=True)
df_generic_saving.head()

,date,bnf_code,pres_stat,name,target_ppu,cost,quantity,target_cost,saving
0,2017-08-01,0408010AEAAACAC,1,Pregabalin_Cap 75mg,0.25,1155933.06,1653845,413461.25,742471.81
1,2016-09-01,0301020S0AAAAAA,1,Glycopyrronium Brom_Inh Cap 55mcg + Dev,0.92,692256.94,25173,23075.25,669181.69
2,2016-08-01,0301020S0AAAAAA,1,Glycopyrronium Brom_Inh Cap 55mcg + Dev,0.92,674547.50,24529,22484.92,652062.58
3,2011-07-01,0803041B0AAAAAA,1,Anastrozole_Tab 1mg,0.28,714975.61,306619,86072.33,628903.28
4,2017-09-01,0408010AEAAACAC,1,Pregabalin_Cap 75mg,0.14,796632.82,1214346,168080.91,628551.91


### Analysis

In [4]:
df_generic_saving.sum()

date           2017-08-012016-09-012016-08-012011-07-012017-0...
bnf_code       0408010AEAAACAC0301020S0AAAAAA0301020S0AAAAAA0...
pres_stat                                                 165074
name           Pregabalin_Cap 75mgGlycopyrronium Brom_Inh Cap...
target_ppu                                            1570361.70
cost                                               6345468237.19
quantity                                             12724357373
target_cost                                        6060642891.02
saving                                              284825346.17
dtype: object

In [5]:
df_2019 = df_generic_saving.loc[(df_generic_saving["date"] >="2019-01-01")] 


In [6]:
df_2019.sum()

date           2019-05-012019-01-012019-03-012019-04-012019-0...
bnf_code       0408010A0AAABAB0408010A0AAABAB0408010A0AAABAB0...
pres_stat                                                   9164
name           Levetiracetam_Tab 500mgLevetiracetam_Tab 500mg...
target_ppu                                              83906.18
cost                                                373365660.71
quantity                                               605301743
target_cost                                         345134072.58
saving                                               28231588.13
dtype: object

In [7]:
df_generic_saving_2019 =  df_2019.loc[(df_generic_saving["pres_stat"] == 1)] ## we restrict to products that should only be prescribed as a VMP e.g. exlcude branded prescribing for narrow TI meds


In [8]:
df_generic_saving_2019.sum()

date           2019-05-012019-01-012019-03-012019-04-012019-0...
bnf_code       0408010A0AAABAB0408010A0AAABAB0408010A0AAABAB0...
pres_stat                                                   6896
name           Levetiracetam_Tab 500mgLevetiracetam_Tab 500mg...
target_ppu                                              79828.28
cost                                                275042266.01
quantity                                               584519973
target_cost                                         244248724.05
saving                                               30793541.96
dtype: object

The saving is approx £30million in first 6 months of 2019